# Preparation of the Jupyter notebook kernel

conda create --name llama python=3.10
   conda activate llama 
    pip install git+https://github.com/huggingface/transformers.git
    pip install git+https://github.com/huggingface/peft.git
  925  pip install git+https://github.com/huggingface/accelerate.git 
  926  pip install -q -U trl
  927  pip install -q trl
  928  pip install -U datasets
  929  pip install -U bitsandbytes 
  930  pip install -U einops
  931  pip install -U wandb
  932  pip install --user ipykernal
  933  pip install --user ipykernel
  934  python -m ipykernel install --user --name=llama

So now you can open jupyter notebook and select the "llama" kernel to run the notebook

In [39]:
# The following command checks how many GPUs are availble in the current system to use
!nvidia-smi

Thu Aug 31 22:00:28 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100 80G...  On   | 00000000:17:00.0 Off |                    0 |
| N/A   43C    P0    81W / 300W |  12263MiB / 81920MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100 80G...  On   | 00000000:65:00.0 Off |                    0 |
| N/A   

## Preprocessing

The purpose of this step is to take the original file downloaded from HPO website, and make a simple JSON file that can be used by LLAMA fine-tuning later on.

In [1]:
import json
with open('/mnt/isilon/wang_lab/wanga18/projects/llama2/hp.json', 'r') as file:
    data = json.load(file)

# Create a dictionary to store disease name to HPO ID mapping
hpo_id_to_disease = {}

# Iterate through each data point and extract HPO ID and "val" data
for item in data["graphs"][0]["nodes"]:
    if "id" in item and "lbl" in item:
        hpo_id = item["id"].split("/")[-1].replace("_", ":")
        #hpo_id = item["id"].split("/")[-1]

        val_data = item["lbl"]
        hpo_id_to_disease[hpo_id] = val_data

In [2]:
hpo_data = [] #create array for training data 
test_data = [] #create array for validation data 

for hpo_id in hpo_id_to_disease:
    disease_name = hpo_id_to_disease[hpo_id] #extract disease and its HPO ID from existing array 
    hpo_data.append({'input':f'The Human Phenotype Ontology term {disease_name} is identified by the HPO ID ', 'output':hpo_id})
    hpo_data.append({'input':f'The HPO ID of {disease_name} corresponds to ', 'output':hpo_id})
    hpo_data.append({'input':f'The HPO term {disease_name} represents ', 'output':hpo_id})
    hpo_data.append({'input':f'The HPO ID of the concept {disease_name} is ', 'output':hpo_id})
    #add sentences to test data (model will not be trained on these sentences)
    test_data.append({'input':f'HPO ID of {disease_name} is ', 'output':hpo_id})
    test_data.append({'input':f'HPO {disease_name} is ', 'output':hpo_id})


In [3]:
# check how many elements (HPO terms) are in the train_data
len(hpo_data)

71616

In [4]:
test_data = test_data[20:9020]

In [5]:
test_data

[{'input': 'HPO ID of Autosomal recessive inheritance is ',
  'output': 'HP:0000007'},
 {'input': 'HPO Autosomal recessive inheritance is ', 'output': 'HP:0000007'},
 {'input': 'HPO ID of Abnormal morphology of female internal genitalia is ',
  'output': 'HP:0000008'},
 {'input': 'HPO Abnormal morphology of female internal genitalia is ',
  'output': 'HP:0000008'},
 {'input': 'HPO ID of Functional abnormality of the bladder is ',
  'output': 'HP:0000009'},
 {'input': 'HPO Functional abnormality of the bladder is ',
  'output': 'HP:0000009'},
 {'input': 'HPO ID of Recurrent urinary tract infections is ',
  'output': 'HP:0000010'},
 {'input': 'HPO Recurrent urinary tract infections is ',
  'output': 'HP:0000010'},
 {'input': 'HPO ID of Neurogenic bladder is ', 'output': 'HP:0000011'},
 {'input': 'HPO Neurogenic bladder is ', 'output': 'HP:0000011'},
 {'input': 'HPO ID of Urinary urgency is ', 'output': 'HP:0000012'},
 {'input': 'HPO Urinary urgency is ', 'output': 'HP:0000012'},
 {'input

In [6]:
print(hpo_data[28:40])

[{'input': 'The Human Phenotype Ontology term Multicystic kidney dysplasia is identified by the HPO ID ', 'output': 'HP:0000003'}, {'input': 'The HPO ID of Multicystic kidney dysplasia corresponds to ', 'output': 'HP:0000003'}, {'input': 'The HPO term Multicystic kidney dysplasia represents ', 'output': 'HP:0000003'}, {'input': 'The HPO ID of the concept Multicystic kidney dysplasia is ', 'output': 'HP:0000003'}, {'input': 'The Human Phenotype Ontology term Mode of inheritance is identified by the HPO ID ', 'output': 'HP:0000005'}, {'input': 'The HPO ID of Mode of inheritance corresponds to ', 'output': 'HP:0000005'}, {'input': 'The HPO term Mode of inheritance represents ', 'output': 'HP:0000005'}, {'input': 'The HPO ID of the concept Mode of inheritance is ', 'output': 'HP:0000005'}, {'input': 'The Human Phenotype Ontology term Autosomal dominant inheritance is identified by the HPO ID ', 'output': 'HP:0000006'}, {'input': 'The HPO ID of Autosomal dominant inheritance corresponds to 

In [7]:
# take a small subset of train data 
train_subset = hpo_data[200:36200]

In [8]:
len(train_subset)

36000

In [9]:
train_subset

[{'input': 'The Human Phenotype Ontology term Hypoplastic male external genitalia is identified by the HPO ID ',
  'output': 'HP:0000050'},
 {'input': 'The HPO ID of Hypoplastic male external genitalia corresponds to ',
  'output': 'HP:0000050'},
 {'input': 'The HPO term Hypoplastic male external genitalia represents ',
  'output': 'HP:0000050'},
 {'input': 'The HPO ID of the concept Hypoplastic male external genitalia is ',
  'output': 'HP:0000050'},
 {'input': 'The Human Phenotype Ontology term Perineal hypospadias is identified by the HPO ID ',
  'output': 'HP:0000051'},
 {'input': 'The HPO ID of Perineal hypospadias corresponds to ',
  'output': 'HP:0000051'},
 {'input': 'The HPO term Perineal hypospadias represents ',
  'output': 'HP:0000051'},
 {'input': 'The HPO ID of the concept Perineal hypospadias is ',
  'output': 'HP:0000051'},
 {'input': 'The Human Phenotype Ontology term Urethral atresia, male is identified by the HPO ID ',
  'output': 'HP:0000052'},
 {'input': 'The HPO I

In [10]:
#dump the newly generated data subset into a JSON file 
with open("train_part1.json", "w") as f: 
   json.dump(train_subset, f)
with open("train_part2.json", "w") as f: 
   json.dump(test_data, f)


## Setup

In [11]:
#import libraries

import pandas as pd
import transformers
import textwrap
from transformers import LlamaTokenizer, LlamaForCausalLM
import os
import sys
from typing import List

from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_int8_training,
)

# import fire
import torch
from datasets import load_dataset
import pandas as pd

import json


DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE

/home/wanga18/miniconda3/envs/llama/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'cuda'

## Model load & data load

In [12]:
#extract the data from the JSON file 
train_data = load_dataset("json", data_files="train_part1.json")
test_data = load_dataset("json", data_files="train_part2.json")

Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 221.24it/s]
Generating train split: 36000 examples [00:00, 148430.22 examples/s]
Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 546.99it/s]
Generating train split: 9000 examples [00:00, 565075.46 examples/s]


In [13]:
CUTOFF_LEN = 512

## Inference

In [14]:
BASE_MODEL = "/mnt/isilon/wang_lab/jingye/projects/llama2/hf/7B"
lora_weights = '/mnt/isilon/wang_lab/wanga18/projects/llama2/experiments_llama2/official_runs/HPO_MODEL/version1'

In [15]:
load_8bit = False

In [16]:
model = LlamaForCausalLM.from_pretrained(
            BASE_MODEL,
            load_in_8bit=load_8bit,
            torch_dtype=torch.float16,
            device_map = "auto"
        )

Loading checkpoint shards: 100%|██████████| 2/2 [00:31<00:00, 15.91s/it]


In [17]:

from peft import PeftModel
model = PeftModel.from_pretrained(
            model,
            lora_weights,
            torch_dtype=torch.float16,
        )


In [18]:
tokenizer = LlamaTokenizer.from_pretrained(BASE_MODEL)

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565, and set the legacy attribute accordingly.


In [19]:
model.config.pad_token_id = tokenizer.pad_token_id = 0  # unk
model.config.bos_token_id = 1
model.config.eos_token_id = 2

In [20]:
from transformers import GenerationConfig
generation_config = GenerationConfig(
            temperature=0.1,
            top_p=0.5,
#             top_k=40,
#             num_beams=4,
#             **kwargs,
#             bad_words_ids = [[2]]
        )

/home/wanga18/miniconda3/envs/llama/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, temperature is set to 0.1 -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset temperature. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/wanga18/miniconda3/envs/llama/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, top_p is set to 0.5 -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset top_p. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [22]:
train_list = train_data['train']
test_list = test_data['train']

train_list[17665]

{'input': 'The HPO ID of Reye syndrome-like episodes corresponds to ',
 'output': 'HP:0006582'}

In [47]:
train_list[35000]

{'output': 'HP:0012112',
 'input': 'The Human Phenotype Ontology term Abnormal circulating corticosterone level is identified by the HPO ID '}

In [48]:
#manually test prompts one by one 
prompt = f"""The HPO ID of the concept Abnormal circulating corticosterone level is HP:"""

prompt = f"""The human phenotype ontology term (HPO) ID for """

In [51]:
inputs = tokenizer(prompt, return_tensors="pt")

In [52]:
input_ids = inputs["input_ids"].to('cuda')

In [54]:
with torch.no_grad():
    generation_output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=60,
    )

In [55]:
generation_output

GreedySearchDecoderOnlyOutput(sequences=tensor([[    1,   450,   379, 13152,  3553,   310,   278,  6964,  1976,  8945,
         18342,  1218, 13979,   293, 15664,   650,  3233,   338,   379, 29925,
         29901, 29900, 29900, 29896, 29906, 29896, 29896, 29906,     2]],
       device='cuda:0'), scores=(tensor([[-6.2188, -2.6230,  9.4531,  ..., -0.8394, -2.6094, -1.4023]],
       device='cuda:0'), tensor([[-4.5430,  0.1001,  8.4531,  ...,  0.9956, -1.6133, -2.4727]],
       device='cuda:0'), tensor([[-5.5938, -9.9609,  3.8008,  ..., -0.2700, -1.3252, -1.9385]],
       device='cuda:0'), tensor([[ -5.9805, -11.6797,   6.0938,  ...,  -1.3252,  -1.2910,  -2.9141]],
       device='cuda:0'), tensor([[ -5.8672, -14.2031,   4.9609,  ...,  -1.1895,   0.4097,  -0.5596]],
       device='cuda:0'), tensor([[ -6.2109, -10.2656,   5.2266,  ...,  -2.8281,  -2.0645,  -0.8936]],
       device='cuda:0'), tensor([[-5.6289, -7.7969,  4.7695,  ..., -2.8320, -5.8477, -1.8711]],
       device='cuda:0'), tenso

In [56]:
s = generation_output.sequences[0]
output = tokenizer.decode(s)
print(output)

<s> The HPO ID of the concept Abnormal circulating corticosterone level is HP:0012112</s>


In [57]:
#code below evaluates model's performance using original training data 

predictionsCorrect = 0 #correct prediction
predictionsIncorrect = 0 #incorrect predictions 

for i in range(0, len(train_list)-1): #loop through all fine-tuned diseases
    disease_name = train_list['input'][i] #get training sentence 
    hpo_id = train_list['output'][i] #get corresponding disease ID (HPO)
    prompt = f"""{disease_name} HP:"""

    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].to('cuda')
    
    with torch.no_grad():
        generation_output = model.generate(
            input_ids=input_ids,
            generation_config=generation_config,
            return_dict_in_generate=True,
            output_scores=True,
            max_new_tokens=60,
        )
    generation_output #generate answer 
    s = generation_output.sequences[0]
    output = tokenizer.decode(s) #decode into string 
    
    print("------------------------")
        
    if hpo_id in output: #check if the output contains the correct HPO ID 
        #print(f"TRUE\n{disease_name} {output}\nANSWER: {hpo_id}\n")
        predictionsCorrect +=1 
    else:
        print(f"FALSE\n{disease_name}{output}\nANSWER: {hpo_id}\n")
        predictionsIncorrect +=1
        
print(f"finished validating on the original training sentences")
print(f"total correct: {totalCorrect}")
print(f"total incorrect: {totalIncorrect}")

------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------


------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------


------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------


------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------


------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------


...

------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
FALSE
The HPO term Decreased fertility in males represents <s> The HPO term Decreased fertility in males represents  HP:0000044</s>
ANSWER: HP:0012041

------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
-----------------------

------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------


------------------------
------------------------
------------------------
------------------------
FALSE
The Human Phenotype Ontology term Cheyne-Stokes respiration is identified by the HPO ID <s> The Human Phenotype Ontology term Cheyne-Stokes respiration is identified by the HPO ID  HP:0012191</s>
ANSWER: HP:0012196

------------------------
FALSE
The HPO ID of Cheyne-Stokes respiration corresponds to <s> The HPO ID of Cheyne-Stokes respiration corresponds to  HP:0012191</s>
ANSWER: HP:0012196

------------------------
FALSE
The HPO term Cheyne-Stokes respiration represents <s> The HPO term Cheyne-Stokes respiration represents  HP:0012191</s>
ANSWER: HP:0012196

------------------------
FALSE
The HPO ID of the concept Cheyne-Stokes respiration is <s> The HPO ID of the concept Cheyne-Stokes respiration is  HP:0012191</s>
ANSWER: HP:0012196

------------------------
------------------------
------------------------
------------------------
------------------------
--------------------

------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------


------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------
------------------------


In [23]:
#this evaluates the model's performance when dealing with prompts it was not trained on 

totalCorrect = 0 #correct predictions 
totalIncorrect = 0 #incorrect predictions 

for i in range(220, len(test_list)-1): #loop through all fine-tuned diseases
    disease_name = test_list['input'][i] #get disease name 
    hpo_id = test_list['output'][i] #get corresponding disease ID (HPO)
    prompt = f"""{disease_name}HP:"""

    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].to('cuda')
    
    with torch.no_grad():
        generation_output = model.generate(
            input_ids=input_ids,
            generation_config=generation_config,
            return_dict_in_generate=True,
            output_scores=True,
            max_new_tokens=60,
        )
    generation_output #generate answer 
    s = generation_output.sequences[0]
    output = tokenizer.decode(s) #decode into string 
    
    print("--------------------------") #seperator line for clarity 
    
    if hpo_id in output: #check if the output contains the correct HPO ID 
        #print(f"TRUE\n{disease_name} {output}\nANSWER: {hpo_id}\n")
        totalCorrect +=1 
    else:
        print(f"FALSE\n{disease_name}{output}\nANSWER: {hpo_id}\n")
        totalIncorrect +=1
        
        
print(f"total correct: {totalCorrect}")
print(f"total incorrect: {totalIncorrect}")

/home/wanga18/miniconda3/envs/llama/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, temperature is set to 0.1 -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset temperature.
  warnings.warn(
/home/wanga18/miniconda3/envs/llama/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, top_p is set to 0.5 -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset top_p.
  warnings.warn(


--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------


KeyboardInterrupt: 

In [ ]:
print(f"finished validating on untrained sentences")
print(f"total correct: {totalCorrect}")
print(f"total incorrect: {totalIncorrect}")